# Section 22: File Staging API Testing

This notebook tests all File Staging API endpoints systematically.

**API Documentation Reference:**  
- File Staging API allows creation and management of files/folders in Vault's file staging
- Supports uploads up to 50MB (regular) and 500GB (resumable uploads)
- Requires 'Application: File Staging: Access' permission
- Different paths for Admin vs non-Admin users

**Endpoints Covered:**
1. **Basic Operations:**
   - List Items at a Path 
   - Download Item Content
   - Create Folder or File
   - Update Folder or File 
   - Delete File or Folder

2. **Resumable Upload Sessions (for files >50MB):**
   - Create Resumable Upload Session
   - Upload to a Session
   - Commit Upload Session
   - List Upload Sessions
   - Get Upload Session Details
   - List File Parts Uploaded to Session
   - Abort Upload Session

**Test Approach:**
- Read-only operations where possible to avoid modifying staging area
- Create minimal test data for operations that require it
- Focus on endpoint accessibility and response structure validation
- Handle permission-based errors gracefully

In [1]:
# Authentication and Setup
import sys
import os
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')

from veevavault.client.vault_client import VaultClient
import requests
import json
from dataclasses import dataclass
from typing import Dict, Any, Optional, List
import time

# Load credentials 
exec(open('/Users/mp/Documents/Code/VeevaTools/veevatools/veevavault/test_credentials.py').read())

# Extract credentials from TEST_VAULTS
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_DNS = vault_config["URL"]
USERNAME = vault_config["username"]
PASSWORD = vault_config["password"]

print(f"🔐 Starting authentication to {VAULT_DNS}...")

# Initialize client and authenticate
client = VaultClient()
auth_response = client.authenticate(
    vaultURL=VAULT_DNS,
    vaultUserName=USERNAME,
    vaultPassword=PASSWORD,
    if_return=True
)

print(f"✅ Authenticated to Vault: {client.vaultURL}")
print(f"🔑 Session ID: {client.session_id[:20]}...")
vault_url = client.vaultURL
session_id = client.session_id
api_version = "v25.2"

🔐 Starting authentication to https://vv-consulting-michael-mastermind.veevavault.com/...
✅ Authenticated to Vault: https://vv-consulting-michael-mastermind.veevavault.com/
🔑 Session ID: 976553B26EB9811B2FF1...


In [2]:
@dataclass
class TestResult:
    """Data class to track individual test results"""
    endpoint: str
    method: str
    success: bool
    status_code: Optional[int] = None
    response_data: Optional[Dict[str, Any]] = None
    error_message: Optional[str] = None
    notes: Optional[str] = None

class FileStagingAPITester:
    """Comprehensive File Staging API testing class"""
    
    def __init__(self, vault_url: str, session_id: str, api_version: str = "v25.2"):
        self.vault_url = vault_url.rstrip('/')
        self.session_id = session_id
        self.api_version = api_version
        self.base_url = f"{self.vault_url}/api/{api_version}/services/file_staging"
        self.headers = {
            'Authorization': session_id,
            'Accept': 'application/json'
        }
        self.test_results: List[TestResult] = []
        
    def _make_request(self, method: str, endpoint: str, **kwargs) -> TestResult:
        """Make HTTP request and return standardized test result"""
        url = f"{self.base_url}{endpoint}"
        
        try:
            # Handle headers properly - merge with defaults
            request_headers = self.headers.copy()
            if 'headers' in kwargs:
                request_headers.update(kwargs.pop('headers'))
            
            response = requests.request(method, url, headers=request_headers, **kwargs)
            
            # Try to parse JSON response
            try:
                response_data = response.json()
            except:
                response_data = {"raw_response": response.text[:500]}
            
            success = response.status_code < 400
            error_msg = None if success else f"HTTP {response.status_code}: {response_data.get('errors', response.text)}"
            
            return TestResult(
                endpoint=endpoint,
                method=method,
                success=success,
                status_code=response.status_code,
                response_data=response_data,
                error_message=error_msg
            )
            
        except Exception as e:
            return TestResult(
                endpoint=endpoint,
                method=method,
                success=False,
                error_message=f"Request failed: {str(e)}"
            )
    
    def test_list_items_at_path(self) -> TestResult:
        """Test GET /items/{path} - List files and folders at a path"""
        print("\\n🔍 Testing List Items at Path...")
        
        # Test root path access (should work for admin users, might fail for non-admin)
        result = self._make_request("GET", "/items/", params={"limit": 10})
        
        if result.success:
            data = result.response_data.get('data', [])
            result.notes = f"Found {len(data)} items at root path"
            print(f"✅ Success: {result.notes}")
            
            # Show sample items
            for item in data[:3]:
                print(f"   - {item.get('kind', 'unknown')}: {item.get('name', 'unnamed')}")
        else:
            print(f"❌ Failed: {result.error_message}")
            # If root fails, might be permission issue for non-admin users
            if "403" in str(result.error_message) or "401" in str(result.error_message):
                result.notes = "Access denied - may require admin permissions or user-specific path"
        
        self.test_results.append(result)
        return result
    
    def test_list_upload_sessions(self) -> TestResult:
        """Test GET /upload - List active upload sessions"""
        print("\\n📤 Testing List Upload Sessions...")
        
        result = self._make_request("GET", "/upload")
        
        if result.success:
            data = result.response_data.get('data', [])
            result.notes = f"Found {len(data)} active upload sessions"
            print(f"✅ Success: {result.notes}")
            
            # Show sample sessions
            for session in data[:3]:
                print(f"   - Session: {session.get('id', 'unknown')[:20]}... - {session.get('path', 'no path')}")
        else:
            print(f"❌ Failed: {result.error_message}")
        
        self.test_results.append(result)
        return result
    
    def test_create_folder(self) -> TestResult:
        """Test POST /items - Create a test folder"""
        print("\\n📁 Testing Create Folder...")
        
        # Create a test folder with timestamp to avoid conflicts
        timestamp = int(time.time())
        folder_name = f"api_test_folder_{timestamp}"
        
        data = {
            'path': f"/{folder_name}",
            'kind': 'folder'
        }
        
        result = self._make_request("POST", "/items", 
                                  headers={'Content-Type': 'application/x-www-form-urlencoded'},
                                  data=data)
        
        if result.success:
            created_path = result.response_data.get('data', {}).get('path', 'unknown')
            result.notes = f"Created folder: {created_path}"
            print(f"✅ Success: {result.notes}")
            # Store created folder path for cleanup
            self.created_folder_path = created_path
        else:
            print(f"❌ Failed: {result.error_message}")
            if "403" in str(result.error_message):
                result.notes = "Permission denied - File Staging: Access permission required"
        
        self.test_results.append(result)
        return result
    
    def test_update_folder(self) -> TestResult:
        """Test PUT /items/{path} - Rename a folder"""
        print("\\n✏️ Testing Update/Rename Folder...")
        
        # Check if we have a created folder to rename
        if not hasattr(self, 'created_folder_path'):
            result = TestResult(
                endpoint="/items/{path}",
                method="PUT",
                success=False,
                error_message="No test folder available to rename",
                notes="Skipped - depends on successful folder creation"
            )
            print(f"⏭️ Skipped: {result.notes}")
            self.test_results.append(result)
            return result
        
        # Rename the folder
        timestamp = int(time.time())
        new_name = f"api_test_folder_renamed_{timestamp}"
        
        # Remove leading slash from path for the endpoint
        folder_path = self.created_folder_path.lstrip('/')
        
        data = {'name': new_name}
        
        result = self._make_request("PUT", f"/items/{folder_path}",
                                  headers={'Content-Type': 'application/x-www-form-urlencoded'},
                                  data=data)
        
        if result.success:
            job_id = result.response_data.get('data', {}).get('job_id', 'unknown')
            result.notes = f"Rename operation initiated, job_id: {job_id}"
            print(f"✅ Success: {result.notes}")
            # Update the folder path for cleanup
            self.created_folder_path = f"/{new_name}"
        else:
            print(f"❌ Failed: {result.error_message}")
        
        self.test_results.append(result)
        return result
    
    def test_delete_folder(self) -> TestResult:
        """Test DELETE /items/{path} - Delete test folder"""
        print("\\n🗑️ Testing Delete Folder...")
        
        # Check if we have a folder to delete
        if not hasattr(self, 'created_folder_path'):
            result = TestResult(
                endpoint="/items/{path}",
                method="DELETE",
                success=False,
                error_message="No test folder available to delete",
                notes="Skipped - no test folder created"
            )
            print(f"⏭️ Skipped: {result.notes}")
            self.test_results.append(result)
            return result
        
        # Remove leading slash from path for the endpoint
        folder_path = self.created_folder_path.lstrip('/')
        
        result = self._make_request("DELETE", f"/items/{folder_path}")
        
        if result.success:
            job_id = result.response_data.get('data', {}).get('job_id', 'unknown')
            result.notes = f"Delete operation initiated, job_id: {job_id}"
            print(f"✅ Success: {result.notes}")
        else:
            print(f"❌ Failed: {result.error_message}")
        
        self.test_results.append(result)
        return result
    
    def test_create_upload_session(self) -> TestResult:
        """Test POST /upload - Create resumable upload session"""
        print("\\n🚀 Testing Create Upload Session...")
        
        timestamp = int(time.time())
        test_file_name = f"api_test_file_{timestamp}.txt"
        
        data = {
            'path': f"/{test_file_name}",
            'size': 1024,  # 1KB test file
            'overwrite': 'true'
        }
        
        result = self._make_request("POST", "/upload",
                                  headers={'Content-Type': 'application/x-www-form-urlencoded'},
                                  data=data)
        
        if result.success:
            session_id = result.response_data.get('data', {}).get('id', 'unknown')
            result.notes = f"Created upload session: {session_id[:20]}..."
            print(f"✅ Success: {result.notes}")
            # Store session ID for potential cleanup
            self.upload_session_id = session_id
        else:
            print(f"❌ Failed: {result.error_message}")
        
        self.test_results.append(result)
        return result
    
    def test_get_upload_session_details(self) -> TestResult:
        """Test GET /upload/{session_id} - Get upload session details"""
        print("\\n📋 Testing Get Upload Session Details...")
        
        if not hasattr(self, 'upload_session_id'):
            result = TestResult(
                endpoint="/upload/{session_id}",
                method="GET",
                success=False,
                error_message="No upload session available",
                notes="Skipped - depends on successful upload session creation"
            )
            print(f"⏭️ Skipped: {result.notes}")
            self.test_results.append(result)
            return result
        
        result = self._make_request("GET", f"/upload/{self.upload_session_id}")
        
        if result.success:
            data = result.response_data.get('data', {})
            uploaded_parts = data.get('uploaded_parts', 0)
            total_size = data.get('size', 0)
            result.notes = f"Session details: {uploaded_parts} parts uploaded, {total_size} bytes total"
            print(f"✅ Success: {result.notes}")
        else:
            print(f"❌ Failed: {result.error_message}")
        
        self.test_results.append(result)
        return result
    
    def test_abort_upload_session(self) -> TestResult:
        """Test DELETE /upload/{session_id} - Abort upload session"""
        print("\\n🛑 Testing Abort Upload Session...")
        
        if not hasattr(self, 'upload_session_id'):
            result = TestResult(
                endpoint="/upload/{session_id}",
                method="DELETE",
                success=False,
                error_message="No upload session available",
                notes="Skipped - no upload session to abort"
            )
            print(f"⏭️ Skipped: {result.notes}")
            self.test_results.append(result)
            return result
        
        result = self._make_request("DELETE", f"/upload/{self.upload_session_id}")
        
        if result.success:
            result.notes = "Upload session aborted successfully"
            print(f"✅ Success: {result.notes}")
        else:
            print(f"❌ Failed: {result.error_message}")
        
        self.test_results.append(result)
        return result
    
    def run_comprehensive_tests(self) -> Dict[str, Any]:
        """Run all File Staging API tests"""
        print("🧪 Starting Comprehensive File Staging API Tests")
        print("=" * 60)
        
        # Run all tests
        test_methods = [
            self.test_list_items_at_path,
            self.test_list_upload_sessions,
            self.test_create_folder,
            self.test_update_folder,
            self.test_delete_folder,
            self.test_create_upload_session,
            self.test_get_upload_session_details,
            self.test_abort_upload_session
        ]
        
        for test_method in test_methods:
            try:
                test_method()
            except Exception as e:
                print(f"❌ Test {test_method.__name__} failed with exception: {e}")
        
        # Calculate summary
        total_tests = len(self.test_results)
        successful_tests = sum(1 for result in self.test_results if result.success)
        
        print("\\n📊 Test Summary")
        print("=" * 40)
        print(f"Total Tests: {total_tests}")
        print(f"Successful: {successful_tests}")
        print(f"Failed: {total_tests - successful_tests}")
        print(f"Success Rate: {(successful_tests/total_tests*100):.1f}%")
        
        return {
            'total_tests': total_tests,
            'successful_tests': successful_tests,
            'failed_tests': total_tests - successful_tests,
            'success_rate': successful_tests/total_tests if total_tests > 0 else 0,
            'test_results': self.test_results
        }

# Initialize the tester
tester = FileStagingAPITester(vault_url, session_id, api_version)
print("📋 File Staging API Tester initialized")
print(f"Base URL: {tester.base_url}")

📋 File Staging API Tester initialized
Base URL: https://vv-consulting-michael-mastermind.veevavault.com/api/v25.2/services/file_staging


In [3]:
# Run comprehensive File Staging API tests
results = tester.run_comprehensive_tests()

🧪 Starting Comprehensive File Staging API Tests
\n🔍 Testing List Items at Path...
✅ Success: Found 6 items at root path
   - file: Michael's Mastermind Vault_English_Field-Labels_8-30-25_23-46-01.csv
   - file: Michael's Mastermind Vault_English_Notification-Messages_8-30-25_23-46-02.csv
   - file: Michael's Mastermind Vault_English_System-Messages_8-30-25_23-46-01.csv
\n📤 Testing List Upload Sessions...
✅ Success: Found 0 active upload sessions
\n📁 Testing Create Folder...
✅ Success: Created folder: /api_test_folder_1756662374
\n✏️ Testing Update/Rename Folder...
✅ Success: Rename operation initiated, job_id: 263804
\n🗑️ Testing Delete Folder...
✅ Success: Delete operation initiated, job_id: unknown
\n🚀 Testing Create Upload Session...
✅ Success: Created upload session: 5ead6a2f3cff3047dfb2...
\n📋 Testing Get Upload Session Details...
✅ Success: Session details: 0 parts uploaded, 1024 bytes total
\n🛑 Testing Abort Upload Session...
✅ Success: Upload session aborted successfully
\n📊 Te

In [4]:
# Display detailed test results
print("\n📋 Detailed Test Results")
print("=" * 60)

for i, result in enumerate(tester.test_results, 1):
    status = "✅ PASS" if result.success else "❌ FAIL"
    print(f"\n{i}. {result.method} {result.endpoint}")
    print(f"   Status: {status}")
    if result.status_code:
        print(f"   HTTP Status: {result.status_code}")
    if result.notes:
        print(f"   Notes: {result.notes}")
    if result.error_message:
        print(f"   Error: {result.error_message}")

print(f"\n🎯 Overall Success Rate: {results['success_rate']:.1%}")


📋 Detailed Test Results

1. GET /items/
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Found 6 items at root path

2. GET /upload
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Found 0 active upload sessions

3. POST /items
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Created folder: /api_test_folder_1756662374

4. PUT /items/api_test_folder_1756662374
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Rename operation initiated, job_id: 263804

5. DELETE /items/api_test_folder_renamed_1756662374
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Delete operation initiated, job_id: unknown

6. POST /upload
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Created upload session: 5ead6a2f3cff3047dfb2...

7. GET /upload/5ead6a2f3cff3047dfb2b7f49398caa8
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Session details: 0 parts uploaded, 1024 bytes total

8. DELETE /upload/5ead6a2f3cff3047dfb2b7f49398caa8
   Status: ✅ PASS
   HTTP Status: 200
   Notes: Upload session aborted successfully

🎯 Ov

In [12]:
# Analyze API coverage and findings
print("\n🔍 File Staging API Coverage Analysis")
print("=" * 50)

# Endpoint coverage
tested_endpoints = {
    'List Items at Path': '/items/{path}',
    'Download Item Content': '/items/content/{path}',  # Not tested (requires specific file)
    'Create Folder or File': '/items',
    'Update Folder or File': '/items/{path}',
    'Delete File or Folder': '/items/{path}',
    'Create Upload Session': '/upload',
    'Upload to Session': '/upload/{session_id}',  # Not tested (requires file data)
    'Commit Upload Session': '/upload/{session_id}',  # Not tested (requires uploaded parts)
    'List Upload Sessions': '/upload',
    'Get Upload Session Details': '/upload/{session_id}',
    'List File Parts': '/upload/{session_id}/parts',  # Not tested (requires uploaded parts)
    'Abort Upload Session': '/upload/{session_id}'
}

print("📊 Endpoint Coverage:")
tested_count = len([r for r in tester.test_results if r.success])
total_endpoints = len(tested_endpoints)
print(f"   - Successfully tested: {tested_count} endpoints")
print(f"   - Total documented: {total_endpoints} endpoints")
print(f"   - Coverage: {(tested_count/total_endpoints*100):.1f}%")

print("\n🔧 Key Findings:")
findings = []
for result in tester.test_results:
    if result.success:
        findings.append(f"✅ {result.endpoint} - Working correctly")
    else:
        findings.append(f"❌ {result.endpoint} - {result.error_message}")

for finding in findings:
    print(f"   {finding}")

print("\n📝 Implementation Notes:")
notes = [
    "File staging operations require 'Application: File Staging: Access' permission",
    "Path access differs between Admin and non-Admin users",
    "Upload sessions support files up to 500GB with multipart upload",
    "Operations like file upload/download require actual file content for full testing",
    "Folder operations work with job-based async processing"
]

for note in notes:
    print(f"   • {note}")


🔍 File Staging API Coverage Analysis
📊 Endpoint Coverage:
   - Successfully tested: 8 endpoints
   - Total documented: 12 endpoints
   - Coverage: 66.7%

🔧 Key Findings:
   ✅ /items/ - Working correctly
   ✅ /upload - Working correctly
   ✅ /items - Working correctly
   ✅ /items/api_test_folder_1756621302 - Working correctly
   ✅ /items/api_test_folder_renamed_1756621303 - Working correctly
   ✅ /upload - Working correctly
   ✅ /upload/1bb208bf03f013dff410f01f81e611f0 - Working correctly
   ✅ /upload/1bb208bf03f013dff410f01f81e611f0 - Working correctly

📝 Implementation Notes:
   • File staging operations require 'Application: File Staging: Access' permission
   • Path access differs between Admin and non-Admin users
   • Upload sessions support files up to 500GB with multipart upload
   • Operations like file upload/download require actual file content for full testing
   • Folder operations work with job-based async processing
